In [ ]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez
from Bio import SeqIO

In [ ]:
Entrez.email = 'greg.clinton@gmail.com'

In [ ]:
# http://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html

# How many genes does e. coli have? What are their names?
# How many species under homo?
# sra vs nucleotide, gene, ref_seq

In [3]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
db = 'nuccore'

In [ ]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
record = Entrez.read(Entrez.esearch(db = dv, term = q))
ids = ','.join(record['IdList'][:5])
handle = Entrez.efetch(db = 'nuccore', id = ids, rettype = 'gb', retmode = 'text')
records = SeqIO.parse(handle, 'gb')

In [ ]:
for record in records:
    print record.name

In [159]:
import requests
import json

email = 'greg.clinton@gmail.com'

def entrez(cgi, path):
    use_json = cgi != 'efetch'
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    url += cgi + '.fcgi?'
    url += '' if path == None else ('%s&' % path) 
    url += 'tool=entrez&email=' + email
    url += '&retmode=%s' % 'json' if use_json else 'text'
    text = requests.get(url).text
    return json.loads(text)[cgi + 'result'] if use_json else text

def einfo(db = None):
    return entrez('einfo', '' if db == None else 'version=2.0&db=%s' % db)

def databases():
    return einfo()['dblist']

def fields(db):
    class Field: pass

    fields = []
    for item in einfo(db)['dbinfo']['fieldlist']:
        field = Field()
        for key, value in item.iteritems():
            setattr(field, key, value)
        fields.append(field)
    return fields

def esearch(db, term):
    res = entrez('esearch', 'db=%s&term=%s&usehistory=y' % (db, term))
    return res['webenv'], res['querykey']

def parse_gb(text):
    return text

def efetch(db, history, rettype = 'gb'):
    env, key = history
    path = 'db=%s&WebEnv=%s&query_key=%s&rettype=%s' % (db, env, key, rettype)
    text = entrez('efetch', path)
    return parse_gb(text) if rettype == 'gb' else text

print efetch('nuccore', esearch('nuccore', q))

LOCUS       NM_001141980            6216 bp    mRNA    linear   PRI 25-AUG-2016
DEFINITION  Homo sapiens tumor protein p53 binding protein 1 (TP53BP1),
            transcript variant 1, mRNA.
ACCESSION   NM_001141980
VERSION     NM_001141980.1  GI:213972635
KEYWORDS    RefSeq.
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 6216)
  AUTHORS   Renaud E, Barascu A and Rosselli F.
  TITLE     Impaired TIP60-mediated H4K16 acetylation accounts for the aberrant
            chromatin accumulation of 53BP1 and RAP80 in Fanconi anemia
            pathway-deficient cells
  JOURNAL   Nucleic Acids Res. 44 (2), 648-656 (2016)
   PUBMED   26446986
  REMARK    GeneRIF: Impaired TIP60-mediated H4K16 acetylation accounts for the
            aberrant chromatin accumulation of 5